In [16]:
import sklearn
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split


from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools

from gensim.models import Word2Vec, Phrases

from sklearn import neighbors
from sklearn import metrics

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
import re
import string

In [17]:
#load data
newsgroups_train = fetch_20newsgroups(subset='train',shuffle=True, remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test',shuffle=True, remove=('headers', 'footers', 'quotes'))

In [22]:
def filter_data():
    #filter data
    # newsgroups_train = fetch_20newsgroups(subset='train',remove=('headers', 'footers', 'quotes'))
    # newsgroups_test = fetch_20newsgroups(subset='test',remove=('headers', 'footers', 'quotes'))
    
    class_names = newsgroups_train.target_names
    print('Training data size::', len(newsgroups_train['data']))
    
    # Printing all the categories
    print('Training categories names::', newsgroups_train.target_names)
    
    # Finding frequency of each category
    targets, frequency = np.unique(newsgroups_train.target, return_counts=True)
    targets_str = np.array(newsgroups_train.target_names)
    
    #Prinnting all frequency
    #print("Print targets, frequency in array::", targets, frequency)
    print(list(zip(targets_str, frequency)))
    
    #Training data class distribution
    #fig=plt.figure(figsize=(10, 5), dpi= 80, facecolor='w', edgecolor='k')
    #plt.bar(targets_str,frequency, color='#ff6961')
    #plt.xticks(rotation=90)
    #plt.title('Class distribution of 20 Newsgroups Training Data')
    #plt.xlabel('News Group')
    #plt.ylabel('Frequency')
    #plt.show()
    
    # Preparing train data
    newsgroups_train_df = pd.DataFrame({'data': newsgroups_train.data, 'target': newsgroups_train.target})

    # Processing the data to remove partial labels

    newsgroups_split_train, newsgroups_split_test = train_test_split(newsgroups_train_df,train_size = 0.3, random_state = 42, shuffle = True)

    newsgroups_split_test = newsgroups_split_train.drop('target', axis = 1)

    newsgroups_train_df = pd.concat([newsgroups_split_train, newsgroups_split_test], axis=0)


    #print(newsgroups_train_df.head())
    # Preparing test data
    newsgroups_test_df = pd.DataFrame({'data': newsgroups_test.data, 'target': newsgroups_test.target})

    newsgroups_test_df = newsgroups_test_df.drop('target',axis = 1)
    print(newsgroups_test_df)
    
    # Text preprocessing steps - remove numbers, captial letters and punctuation
    newsgroups_train_df.data.str.replace('[^a-zA-Z]', '')
    newsgroups_test_df.data.str.replace('[^a-zA-Z]', '')
    newsgroups_train_df = newsgroups_train_df.data.str.lower()
    newsgroups_test_df = newsgroups_test_df.data.str.lower()
    print(newsgroups_train_df.head())
    

    return newsgroups_train, newsgroups_test, class_names

In [19]:
def bow_features(train_data, test_data):
    # Bag-of-words representation
    bow_vectorize = CountVectorizer()
    bow_train = bow_vectorize.fit_transform(train_data.data) #bag-of-word features for training data
    bow_test = bow_vectorize.transform(test_data.data)
    feature_names = bow_vectorize.get_feature_names() #converts feature index to the word it represents.
    shape = bow_train.shape
    print('{} train data points.'.format(shape[0]))
    print('{} feature dimension.'.format(shape[1]))
    print('Most common word in training set is "{}"'.format(feature_names[bow_train.sum(axis=0).argmax()]))
    return bow_train, bow_test, feature_names

In [20]:
def tf_idf_features(train_data, test_data):
    # Bag-of-words representation
    tf_idf_vectorize = TfidfVectorizer(ngram_range=(1, 3), stop_words='english')
    tf_idf_train = tf_idf_vectorize.fit_transform(train_data.data) #bag-of-word features for training data
    feature_names = tf_idf_vectorize.get_feature_names() #converts feature index to the word it represents.
    tf_idf_test = tf_idf_vectorize.transform(test_data.data)
    print(tf_idf_train)
    return tf_idf_train, tf_idf_test, feature_names

In [37]:
def w2v(feature_names):
  feature_names = list(feature_names)
  n = 5
  list_of_lists = [feature_names[i:i + n] for i in range(0, len(feature_names), n)]
  print(list_of_lists)

  model = Word2Vec(list_of_lists, size=300, min_count=1, workers=4)
  # model['browser']
  # model.similar_by_word('anarchist')

  return model

In [ ]:
if __name__ == '__main__':
    train_data, test_data, class_names = filter_data()

    # #Count Vectorization
    # train_bow, test_bow, feature_names = bow_features(train_data, test_data)
    
    # # TF-idf
    # train_bow_tf_idf, test_bow_tf_idf, feature_names_tf_idf = tf_idf_features(train_data, test_data)
    # print('------')
    # print(train_bow_tf_idf)
    # print('-----')
    # print(feature_names)

    # Word2Vec

    word_embedding = w2v(feature_names)

    print(word_embedding.wv.similar_by_word('browser'))

